In [ ]:
filenames = [
    # 'O2 Grouper Pipeline - Task List.xlsx',
    # 'P2 v3 Authorization_DS manager - list of tasks.xlsx',
    'P2 v3 Replatforming Tasks List.xlsx',
    # 'P2 testing tasks.xlsx',
]


In [ ]:
import tools.dev_o2_excel_ingestor as o2

for filename in filenames:
    print("Ingesting: ", filename)
    rdf = o2.generate_rdf_from_xlsx('./tools/', filename)
    o2.send_rdf_to_dgraph('localhost:9080', rdf)
    print("Finished ingestion: ", filename)



In [5]:
import tools.lib.pmtx_client.query_baselines as rb
import requests
import json
import pandas as pd


url = 'http://localhost:8080/graphql'


query_project_filter = {
    "or": [
        {"name": {"allofterms": "P2 v3 Replatforming Tasks List.xlsx"}},
        # {"name": {"allofterms": "O2 Grouper Pipeline - Task List.xlsx"}},
        # {"name": {"allofterms": "P2 v3 Authorization_DS manager - list of tasks.xlsx"}}
    ]
}


normalized_baseline = rb.request_and_normalize_baselines_from_pmtx(url, query_project_filter)
dfp, dfd = rb.baseline_to_pandas_df(normalized_baseline)



query = '''
    query ($projects_ids:[ID!]) {
        queryProject (filter: {and: [{id: $projects_ids} ]}) {
            id
            customFields
        }
    }
'''

r = requests.post(url=url, json={"query": query, "variables": {"projects_ids": dfp.project_id.to_list()}})

project_details = r.json()['data']['queryProject']

for project in project_details:
    if project['customFields'] is not None:
        details = json.loads(project['customFields'])
        project['sprint'] = (int(details['sprint'].split(' ')[1]) if details['sprint'].split(' ')[1].isdigit() else None) if 'sprint' in details else None
    else: 
        project['sprint'] = None
    del project['customFields']

project_details = pd.DataFrame(project_details)
dfp = dfp.merge(project_details, how='left', left_on='project_id', right_on='id')
dfp.drop(columns=['id'], inplace=True)


print("Finished.")


Finished.


In [ ]:
import tools.lib.task_assignee_estimators.solver_base as base

est = base.SolverBase(dfp,dfd)

est.config()
est.initialize()

est.find_incorrect_dependencies_FS()

# est.lp

# dfp[dfp.start.notnull() & dfp.finish.notnull()]

In [ ]:
from time import time
import pandas as pd

import tools.lib.task_assignee_estimators.primitive_estimation as estimator




solver = estimator.ProposeAssigment()

start_date = dfp.start.min()


projects = dfp.copy(deep=True)
dependencies = dfd.copy(deep=True)

solver.av = estimator.create_availability_calendar(number_of_users=9)
solver.av['project_id'] = None

algo_time_start = time()

for sprint in [6,7,8,9,10]:
    solver.projects = projects[(projects.sprint < sprint)]
    # print(solver.projects[solver.projects.sprint == 6].shape[0])
    solver.dependencies = dependencies
    solver.av[solver.av.project_id.notnull()]

    # # Calculation part
    solver.initialize(start_date)
    solver.av[solver.av.project_id.notnull()]
    # x1 = solver.av.copy(deep=True)

    # finish_date = solver.assign_projects_from_last_task_based_on_infinite_resources(partial_update=True, partial_update_from=pd.Timestamp('2021-01-01', tz='UTC'), one_worker_per_project=True)
    finish_date = solver.assign_projects_by_start_based_on_infinite_resources(partial_update=True, partial_update_from=pd.Timestamp('2021-01-01', tz='UTC'), one_worker_per_project=True)

    solver.av[solver.av.project_id.notnull()]
    # x2 = solver.av.copy(deep=True)

    solver.update_projects()
    solver.av[solver.av.project_id.notnull()]


    for index, row in solver.projects.iterrows(): # TODO: find better way to update
        projects.loc[projects.project_id == row['project_id'], 'start'] = solver.projects[solver.projects.project_id == row['project_id']].start.iloc[0]
        projects.loc[projects.project_id == row['project_id'], 'finish'] = solver.projects[solver.projects.project_id == row['project_id']].finish.iloc[0]


    break

# ## resolve rest (NULLs)
# solver.projects = projects
# # print(solver.projects[solver.projects.sprint == 6].shape[0])
# solver.dependencies = dependencies

# # # Calculation part
# solver.initialize(start_date)

# finish_date = solver.assign_projects_by_start_based_on_infinite_resources(partial_update=True, partial_update_from=pd.Timestamp('2021-01-01', tz='UTC'), one_worker_per_project=True)

# solver.update_projects()

# # projects[(projects.sprint < sprint), 'start'] = solver.projects
# for index, row in solver.projects.iterrows(): # TODO: find better way to update
#     projects.loc[projects.project_id == row['project_id'], 'start'] = solver.projects[solver.projects.project_id == row['project_id']].start.iloc[0]
#     projects.loc[projects.project_id == row['project_id'], 'finish'] = solver.projects[solver.projects.project_id == row['project_id']].finish.iloc[0]


algo_time_finish = time()

print('Project finish timestamp: ' + str(finish_date))
print('Calculation time [s]: ' + str(algo_time_finish - algo_time_start))
print('Unassigned workers time during project: ' + str((solver.av[solver.av.project_id.isnull() & (solver.av.start <= finish_date)].finish - solver.av[solver.av.project_id.isnull() & (solver.av.start <= finish_date)].start).sum()))

print("Finished.")




In [ ]:
sprt = 6
print("min: ", solver.projects[solver.projects.sprint == sprt].start.min())
print("min: ", solver.projects[solver.projects.sprint == sprt].start.max())

In [7]:
import tools.lib.pmtx_client.mutate_baselines as mb
import datetime

solver.projects.drop(columns=['sprint'], inplace=True)

root_id = "0x2e8a"
baseline_name = "X last finish solver - from actual times " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")


baseline_id = mb.add_baseline_and_return_id(url, baseline_name, root_id)
mb.add_project_baseline(url, solver.projects, baseline_id, root_id)
mb.modify_project_baseline_predecessors(url, solver.dependencies, baseline_id)
mb.add_resource_baseline(url, solver.av, baseline_id)

print("Finished.")

d:\pmt_x\tools\lib\pmtx_client\mutate_baselines.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  children['parent_id'] = root_id
{'data': {'addProjectBaseline': {'projectBaseline': [{'id': '0x4dde', 'project': {'id': '0x2e8a'}, 'baseline': {'id': '0x4d19'}}]}}, 'extensions': {'touched_uids': 5724, 'tracing': {'version': 1, 'startTime': '2021-05-03T21:12:13.1994288Z', 'endTime': '2021-05-03T21:12:13.7417352Z', 'duration': 542306400, 'execution': {'resolvers': [{'path': ['addProjectBaseline'], 'parentType': 'Mutation', 'fieldName': 'addProjectBaseline', 'returnType': 'AddProjectBaselinePayload', 'startOffset': 14096100, 'duration': 528114700, 'dgraph': [{'label': 'mutation', 'startOffset': 71376500, 'duration': 418766600}, {'label': 'query', 's

In [ ]:
from time import time
import pandas as pd

import tools.lib.task_assignee_estimators.primitive_estimation as estimator






solver = estimator.ProposeAssigment()

start_date = dfp.start.min()

solver.projects = dfp.copy(deep=True)
solver.dependencies = dfd.copy(deep=True)
# solver.av = pd.DataFrame([], columns=['user_id', 'start', 'finish'])
solver.av = estimator.create_availability_calendar(number_of_users=9)
# solver.av = expander.create_availability_frame()
solver.av['project_id'] = None
print(solver.av.shape[0])

# Calculation part

solver.initialize(start_date)


algo_time_start = time()


finish_date = solver.assign_projects_infinite_resources()
# finish_date = solver.assign_projects_to_resources_first_free(one_worker_per_project=True)
# finish_date = solver.assign_projects_by_start_based_on_infinite_resources(one_worker_per_project=True)
# finish_date = solver.assign_projects_by_start_based_on_infinite_resources(partial_update=True, partial_update_from=pd.Timestamp('2021-01-01', tz='UTC'), one_worker_per_project=True)

print(solver.av.shape[0])
solver.update_projects()

algo_time_finish = time()

print('Project finish timestamp: ' + str(finish_date))
print('Calculation time [s]: ' + str(algo_time_finish - algo_time_start))
# print('Unassigned workers time during project: ' + str((solver.av[solver.av.project_id.isnull() & (solver.av.start <= finish_date)].finish - solver.av[solver.av.project_id.isnull() & (solver.av.start <= finish_date)].start).sum()))

print("Finished.")

In [6]:
from time import time
import pandas as pd

import tools.lib.task_assignee_estimators.last_finish_solver as estimator



solver = estimator.LastFinishSolver(dfp,dfd)

solver.config()


solver.av = estimator.LastFinishSolver.create_availability_calendar(start_time=pd.Timestamp('2021-05-02 21:04:56.092887+0000'), number_of_users=5)

solver.av['project_id'] = None
print(solver.av.shape[0])

# Calculation part

solver.initialize()


algo_time_start = time()


finish_date = solver.allocate_projects(create_plan_from_scratch=False)
# finish_date = solver.assign_projects_to_resources_first_free(one_worker_per_project=True)
# finish_date = solver.assign_projects_by_start_based_on_infinite_resources(one_worker_per_project=True)
# finish_date = solver.assign_projects_by_start_based_on_infinite_resources(partial_update=True, partial_update_from=pd.Timestamp('2021-01-01', tz='UTC'), one_worker_per_project=True)

print(solver.av.shape[0])
solver.update_projects()

algo_time_finish = time()

print('Project finish timestamp: ' + str(finish_date))
print('Calculation time [s]: ' + str(algo_time_finish - algo_time_start))
# print('Unassigned workers time during project: ' + str((solver.av[solver.av.project_id.isnull() & (solver.av.start <= finish_date)].finish - solver.av[solver.av.project_id.isnull() & (solver.av.start <= finish_date)].start).sum()))

print("Finished.")

1075
Projects in tree: {'0x2c9d', '0x2cdc', '0x2dc6', '0x2f3f', '0x2ecf', '0x2e0d', '0x2b8a', '0x2d2c', '0x2c62', '0x2b96', '0x2d8d', '0x2e59', '0x2d1f', '0x2ed2', '0x2c0e', '0x2f7e', '0x3058', '0x2b79', '0x2e81', '0x2ca7', '0x2e2a', '0x2ef5', '0x2b86', '0x2d5a', '0x2bb0', '0x2c26', '0x30fe', '0x2eb5', '0x2fde', '0x2fe3', '0x2f39', '0x2fe8', '0x2d21', '0x2ed7', '0x2f21', '0x2c3e', '0x2d71', '0x2e1e', '0x2ccc', '0x306d', '0x2c7a', '0x2ebe', '0x2e5a', '0x2e80', '0x2fda', '0x2e88', '0x2ea9', '0x2f3d', '0x2c4a', '0x30fd', '0x2ee4', '0x2e48', '0x3050', '0x2b83', '0x3084', '0x3103', '0x2fe7', '0x3081', '0x2cbb', '0x2d54', '0x30bc', '0x2d5b', '0x300a', '0x2d31', '0x2fd6', '0x3057', '0x2e5b', '0x30a5', '0x2f66', '0x2f16', '0x30f2', '0x306c', '0x2b97', '0x2c37', '0x2f00', '0x2d59', '0x3034', '0x2c63', '0x2cfc', '0x2d52', '0x2c17', '0x308f', '0x2d24', '0x2f57', '0x2e9f', '0x2b7d', '0x2c12', '0x2f1e', '0x2cd5', '0x2d2d', '0x2e7c', '0x2f4e', '0x30cb', '0x3073', '0x2fba', '0x2dd1', '0x2cad', '0x2c5